In [ ]:
import os
import pandas as pd
from urllib.parse import unquote

# 현재 디렉토리의 모든 .csv 파일 리스트 가져오기
csv_files = [file for file in os.listdir('.') if file.endswith('.csv')]

# 각 파일 읽기
dataframes = []  # 배열로 변경
for file in csv_files:
    try:
        df = pd.read_csv(file)
        dataframes.append((file, df))  # 파일 이름과 데이터프레임을 튜플로 저장
        print(f"{file} 읽기 완료")
    except Exception as e:
        print(f"{file} 읽기 실패: {e}")

# 확인: 읽어온 데이터프레임 목록 출력
print(f"총 {len(dataframes)}개의 CSV 파일을 읽어왔습니다.")

In [ ]:
for file, df in dataframes:
    print(f"{len(df)} rows")

In [ ]:
def calculate_correct_answers(file_df_tuple):
    """
    데이터프레임에서 'answer'와 'user_choice' 열이 같은 값을 세는 함수.
    """
    file, df = file_df_tuple
    if 'answer' in df.columns and 'user_choice' in df.columns:  # 열 확인
        return (file, (df['answer'].str.lower() == df['user_choice'].str.lower()).sum())
    else:
        return (file, None)  # 열이 없으면 None 반환

# map을 사용하여 데이터프레임 리스트에 적용
results = map(calculate_correct_answers, dataframes)

# 결과를 리스트로 변환
results = list(results)

# count 기준으로 정렬 (None 값은 맨 뒤로 이동)
sorted_results = sorted(results, key=lambda x: (x[1] is None, x[1]), reverse=True)

# 결과 출력
for file, count in sorted_results:
    if count is not None:
        print(f"{file}: 정답 수 = {count}, 비율 = {int(count / 225 * 100)}%")
    else:
        print(f"{file}: 'answer' 또는 'user_choice' 열이 없습니다.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

# 경고 무시 설정
warnings.filterwarnings('ignore')

# 유효한 count 값들만 추출
valid_counts = [count for _, count in results if count is not None]

# Seaborn을 사용하여 count 분포 시각화
plt.figure(figsize=(10, 6))
sns.histplot(valid_counts, bins=20, kde=True)

# 그래프 제목과 레이블 설정
plt.title('Count Distribution of Correct Answers')
plt.xlabel('Number of Correct Answers')
plt.ylabel('Frequency')

# 그래프 출력
plt.show()
